In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
import time
import matplotlib.pyplot as plt 
import tensorflow as tf

import os
os.chdir('../')

import data
import utils

In [8]:
directory = 'output'
data_directory = 'data'
aggregation = 'day'
to_drop_stations = ['(40000) cable portal tunal', '(40001) juan pablo ii','(40002) manitas', '(40003) mirador del paraiso']

settings = data.read_settings()
train, test = data.train_test_data(settings)
train = train.drop(columns= to_drop_stations)
test = test.drop(columns= to_drop_stations)

stations = list(test.columns[test.columns.str.contains('\(')])
# test = test.iloc[31:]
train_mean = train[stations].mean()
train_std = train[stations].std()

# train[stations] = train[stations].apply(np.log1p)
# test[stations] = test[stations].apply(np.log1p)
train[stations] = (train[stations]-train_mean)/train_std
test[stations] = (test[stations]-train_mean)/train_std

train = train.astype(float)
test =test.astype(float)

# # train, test = data.split_data(data_directory, aggregation = aggregation)
# dates = pd.to_datetime(test.reset_index()[['year', 'month', 'day']])
# dates = dates.reset_index(drop = True)

In [9]:
df = pd.concat((train, test))
stations = list(df.columns[df.columns.str.contains('\(')])

In [32]:
test

dayofweek  weekday  (02000) cabecera autopista norte  \
year month day                                                         
2018 8     1          2.0      1.0                          0.103184   
           2          3.0      1.0                          0.136305   
           3          4.0      1.0                          0.161620   
           4          5.0      0.0                         -0.816154   
           5          6.0      0.0                         -1.706449   
...                   ...      ...                               ...   
2021 4     27         1.0      1.0                         -1.686267   
           28         2.0      1.0                         -2.739776   
           29         3.0      1.0                         -2.065190   
           30         4.0      1.0                         -2.176015   
     5     1          5.0      0.0                         -3.112757   

                (02001) centro comercial santa fe  (02101) toberin  \
year month day                                                       
2018 8     1                             0.560223         1.159570   
           2                             0.618074         1.129839   
           3                             0.834566         1.138930   
           4                            -0.542292        -0.462144   
           5                            -1.767965        -1.550095   
...                                           ...              ...   
2021 4     27                           -1.834044        -1.117182   
           28                           -2.508717        -1.925686   
           29                           -2.071105        -1.504536   
           30                           -2.185522        -1.540791   
     5     1                            -2.758891        -2.163210   

                (02102) calle 161  (02103) mazuren  (02104) calle 146  \
year month day                                                          
2018 8     1             0.070551         0.171353           1.101251   
           2             0.152985         0.152992           1.046479   
           3             0.182709         0.235155           1.192879   
           4            -0.892896        -0.824703          -0.320261   
           5            -1.724369        -1.649088          -1.293529   
...                           ...              ...                ...   
2021 4     27           -1.486182        -1.515975          -1.189787   
           28           -2.126234        -1.968561          -1.850292   
           29           -1.681170        -1.665842          -1.407851   
           30           -1.827807        -1.726661          -1.494701   
     5     1            -2.278024        -2.056691          -2.015120   

                (02105) calle 142  (02200) alcala  ...  (12000) puente aranda  \
year month day                                     ...                          
2018 8     1             0.714885        0.784215  ...               0.462174   
           2             0.737243        0.775705  ...               0.291258   
           3             0.754880        0.820235  ...               0.462174   
           4            -0.601765       -0.638841  ...              -0.667829   
           5            -1.772120       -1.761325  ...              -1.679655   
...                           ...             ...  ...                    ...   
2021 4     27           -1.399638       -1.173390  ...              -0.787959   
           28           -2.151926       -2.236106  ...              -2.119155   
           29           -1.620429       -1.530070  ...              -1.391538   
           30           -1.719404       -1.670584  ...              -1.463812   
     5     1            -2.336577       -2.387110  ...              -2.256865   

                (12001) carrera 43  (12002) cds - carrera 32  \
year month day                                                 
2018 8     1              1.062667  

In [25]:
input_width = 14
label_width = 7
shift = 7
num_features = 147
w = data.WindowGenerator(input_width, label_width, shift, df, label_columns= stations)

In [27]:
len(w.label_columns)

147

In [11]:
for input, label in w.test.take(1):
    print('Input')
    print (input.shape)
    print (input[0])
    print ("")
    print ('Label')
    print( label.shape)
    print( label[0])

Input
(32, 14, 149)
tf.Tensor(
[[ 2.          1.         -0.12419809 ...  0.90571445 -0.24333695
   0.71901315]
 [ 3.          1.         -0.02198699 ...  0.87895846 -0.2133726
   0.7891512 ]
 [ 4.          1.         -1.7695262  ... -1.1442354  -1.4871923
  -0.6183388 ]
 ...
 [ 6.          0.         -1.6538488  ... -1.1441348  -1.4736938
  -0.77910477]
 [ 0.          1.         -0.02504594 ...  1.0697708   0.02174696
   0.556671  ]
 [ 1.          1.         -0.03264057 ...  1.0983373   0.13968968
   0.7828467 ]], shape=(14, 149), dtype=float32)

Label
(32, 7, 147)
tf.Tensor(
[[ 0.10318383  0.56022286  1.15957    ...  1.2288984   0.19559759
   0.6693648 ]
 [ 0.13630487  0.61807406  1.1298392  ...  1.1729723   0.27697042
   1.0429093 ]
 [ 0.16162032  0.83456594  1.1389296  ...  1.0307432   0.33441004
   0.521996  ]
 ...
 [-1.7064486  -1.7679652  -1.550095   ... -1.1442354  -1.4843203
  -0.7058144 ]
 [ 0.05248262  0.36867124  0.9545564  ...  0.61019176  0.08694094
   0.7993961 ]
 [-1.81

In [16]:
def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=100,
                    #   validation_data=window.val,
                      callbacks=[early_stopping],
                      verbose = False)
  return history


In [18]:
def prediction_dict(station, prediction):
    """ Saves a <station_name>.json file in the path directory. 
    
    Parameters:
    ------------
    path: 'str'. Directory path to save prediction 
    station_name: 'str'. Name of the station.
    prediction: array-like. Prediction array of the shape (n, OUT STEPS)
    """
    dict_ = {}
    dict_['name'] = station
    dict_['prediction'] = prediction.tolist()
    return dict_

In [ ]:
        conv_model = tf.keras.Sequential([
        tf.keras.layers.Lambda(lambda x: x[:, -self.conv_widht:, :]),
        tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(self.conv_widht)),
        tf.keras.layers.Dense(self.forecast_window*num_features,
                          kernel_initializer=tf.initializers.zeros()),
        tf.keras.layers.Reshape([self.forecast_window, num_features])])

        history = compile_and_fit(conv_model, window)

In [20]:
## CNN Network 
model = tf.keras.Sequential([
                tf.keras.layers.Lambda(lambda x: x[:, -input_width:, :]),
                tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(input_width)),
                tf.keras.layers.Dense(shift*num_features,
                          kernel_initializer=tf.initializers.zeros()),
                tf.keras.layers.Reshape([shift, num_features])])

history = compile_and_fit(model, w)
r = model.predict(w.test)
np_r = np.array(r)

for i in range(len(stations)):
    station_name = stations[i]
    station_prediction = np_r[:,:,i]
    station_prediction = (station_prediction *  train_std[i]) + train_mean[i]
    dict_ = prediction_dict(station_name, station_prediction)
    station_path = os.path.join('output/day/cnn', 'multioutput', station_name + '.json')
    utils.save_json(station_path, dict_)

In [21]:
## Dense Network 
# model = tf.keras.Sequential([
#                 tf.keras.layers.Lambda(lambda x: x[:, -input_width:, :]),
#                 tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(input_width)),
#                 tf.keras.layers.Dense(shift*num_features,
#                           kernel_initializer=tf.initializers.zeros()),
#                 tf.keras.layers.Reshape([shift, num_features])])

model = tf.keras.Sequential([
            tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(shift*num_features,
            kernel_initializer=tf.initializers.zeros()),
            tf.keras.layers.Reshape([shift, num_features])
            ])

history = compile_and_fit(model, w)
r = model.predict(w.test)
np_r = np.array(r)

for i in range(len(stations)):
    station_name = stations[i]
    station_prediction = np_r[:,:,i]
    station_prediction = (station_prediction *  train_std[i]) + train_mean[i]
    dict_ = prediction_dict(station_name, station_prediction)
    station_path = os.path.join('output/day/dense', 'multioutput', station_name + '.json')
    utils.save_json(station_path, dict_)

In [22]:
## LSTM Network 
model = tf.keras.Sequential([
            tf.keras.layers.LSTM(32, return_sequences=False),
            tf.keras.layers.Dense(shift*num_features,
            kernel_initializer=tf.initializers.zeros()),
            # Shape => [batch, out_steps, features].
            tf.keras.layers.Reshape([shift, num_features])
            ])

history = compile_and_fit(model, w)
r = model.predict(w.test)
np_r = np.array(r)

for i in range(len(stations)):
    station_name = stations[i]
    station_prediction = np_r[:,:,i]
    station_prediction = (station_prediction *  train_std[i]) + train_mean[i]
    dict_ = prediction_dict(station_name, station_prediction)
    station_path = os.path.join('output/day/lstm', 'multioutput', station_name + '.json')
    utils.save_json(station_path, dict_)

In [48]:
np_r = np.array(r)

In [53]:
np_r.shape

(999, 7, 147)

In [57]:
train_std[2]

9350.602778182145

In [19]:
def prediction_dict(station, prediction):
    """ Saves a <station_name>.json file in the path directory. 
    
    Parameters:
    ------------
    path: 'str'. Directory path to save prediction 
    station_name: 'str'. Name of the station.
    prediction: array-like. Prediction array of the shape (n, OUT STEPS)
    """
    dict_ = {}
    dict_['name'] = station
    dict_['prediction'] = prediction.tolist()
    return dict_

In [58]:
for i in range(len(stations)):
    station_name = stations[i]
    station_prediction = np_r[:,:,i]
    station_prediction = (station_prediction *  train_std[i]) + train_mean[i]
    dict_ = prediction_dict(station_name, station_prediction)
    station_path = os.path.join('output', 'multioutput', station_name + '.json')
    utils.save_json(station_path, dict_)

In [9]:
for input, labels in w.test.take(1):
    print (labels[0])
    prediction = model(input)

tf.Tensor(
[[11.423722   9.465757  10.34403   ...  0.         0.         0.       ]
 [11.433969   9.483036  10.335043  ...  0.         0.         0.       ]
 [11.4417305  9.545169  10.337799  ...  0.         0.         0.       ]
 ...
 [10.596584   8.257126   8.654169  ...  0.         0.         0.       ]
 [11.407831   9.406319  10.280348  ...  0.         0.         0.       ]
 [10.519942   8.264106   8.768886  ...  0.         0.         0.       ]], shape=(7, 151), dtype=float32)


In [10]:
y_hat = np.array(prediction[0])
y_hat = np.expm1(y_hat)
y_hat

array([[60603.535 ,  7982.264 , 20080.814 , ...,     0.    ,     0.    ,
            0.    ],
       [60083.125 ,  7755.248 , 19697.45  , ...,     0.    ,     0.    ,
            0.    ],
       [59478.684 ,  7627.4214, 19316.98  , ...,     0.    ,     0.    ,
            0.    ],
       ...,
       [58837.215 ,  7534.113 , 18818.1   , ...,     0.    ,     0.    ,
            0.    ],
       [59764.234 ,  7787.187 , 19507.133 , ...,     0.    ,     0.    ,
            0.    ],
       [60465.33  ,  8026.996 , 20043.904 , ...,     0.    ,     0.    ,
            0.    ]], dtype=float32)

In [108]:
target = np.array(test.iloc[:7,:][stations])

In [109]:
target.shape

(7, 151)

In [113]:
ape = (target - y_hat)
np.mean(np.abs(ape))

12789.222169841934

In [71]:
np.float(target)

TypeError: only size-1 arrays can be converted to Python scalars

In [68]:
np.mean(np.arctan(np.abs(ape)))

nan

In [76]:
test.head(50)

dayofweek  weekday  (02000) cabecera autopista norte  \
year month day                                                         
2018 9     1            5        0                           72815.0   
           2            6        0                           44299.0   
           3            0        1                           98402.0   
           4            1        1                           99245.0   
           5            2        1                           94653.0   
           6            3        1                           88767.0   
           7            4        1                          114353.0   
           8            5        0                           71938.0   
           9            6        0                           42040.0   
           10           0        1                           97299.0   
           11           1        1                           97070.0   
           12           2        1                           97777.0   
           13           3        1                           96688.0   
           14           4        1                           99669.0   
           15           5        0                           72017.0   
           16           6        0                           44611.0   
           17           0        1                           96041.0   
           18           1        1                           97642.0   
           19           2        1                           96392.0   
           20           3        1                           95912.0   
           21           4        1                          100329.0   
           22           5        0                           70395.0   
           23           6        0                           42390.0   
           24           0        1                           95976.0   
           25           1        1                           95837.0   
           26           2        1                           96557.0   
           27           3        1                           96045.0   
           28           4        1                           98685.0   
           29           5        0                           71980.0   
           30           6        0                           44885.0   
     10    1            0        1                           96330.0   
           2            1        1                           97675.0   
           3            2        1                           96484.0   
           4            3        1                           93059.0   
           5            4        1                           99293.0   
           6            5        0                           72100.0   
           7            6        0                           46439.0   
           8            0        1                           95647.0   
           9            1        1                           95983.0   
           10           2        1                           91099.0   
           11           3        1                           94743.0   
           12           4        1                           93911.0   
           13           5        0                           65757.0   
           14           6        0                           43366.0   
           15           0        1                           43692.0   
           16           1        1                          100558.0   
           17           2        1                           92887.0   
           18           3        1                           95866.0   
           19           4        1                           98413.0   
           20           5        0                           73063.0   

                (02001) centro comercial santa fe  (02101) toberin  \
year month day                                                       
2018 9     1                               9480.0          16920.0   
           2                               3874.0           6

In [62]:
first_p = prediction[0]
first_p

array([[100712.47  ,  13512.894 ,  31836.291 , ...,      0.    ,
             0.    ,      0.    ],
       [ 97746.38  ,  12233.    ,  29575.758 , ...,      0.    ,
             0.    ,      0.    ],
       [ 83996.2   ,  10309.992 ,  23757.164 , ...,      0.    ,
             0.    ,      0.    ],
       ...,
       [ 50539.34  ,   5353.0674,  11443.03  , ...,      0.    ,
             0.    ,      0.    ],
       [ 88429.96  ,  10272.544 ,  24317.74  , ...,      0.    ,
             0.    ,      0.    ],
       [ 99531.39  ,  12635.962 ,  29924.055 , ...,      0.    ,
             0.    ,      0.    ]], dtype=float32)

In [60]:
test.head(50)#iloc[:7,:]

dayofweek  weekday  (02000) cabecera autopista norte  \
year month day                                                         
2018 9     1            5        0                           72815.0   
           2            6        0                           44299.0   
           3            0        1                           98402.0   
           4            1        1                           99245.0   
           5            2        1                           94653.0   
           6            3        1                           88767.0   
           7            4        1                          114353.0   
           8            5        0                           71938.0   
           9            6        0                           42040.0   
           10           0        1                           97299.0   
           11           1        1                           97070.0   
           12           2        1                           97777.0   
           13           3        1                           96688.0   
           14           4        1                           99669.0   
           15           5        0                           72017.0   
           16           6        0                           44611.0   
           17           0        1                           96041.0   
           18           1        1                           97642.0   
           19           2        1                           96392.0   
           20           3        1                           95912.0   
           21           4        1                          100329.0   
           22           5        0                           70395.0   
           23           6        0                           42390.0   
           24           0        1                           95976.0   
           25           1        1                           95837.0   
           26           2        1                           96557.0   
           27           3        1                           96045.0   
           28           4        1                           98685.0   
           29           5        0                           71980.0   
           30           6        0                           44885.0   
     10    1            0        1                           96330.0   
           2            1        1                           97675.0   
           3            2        1                           96484.0   
           4            3        1                           93059.0   
           5            4        1                           99293.0   
           6            5        0                           72100.0   
           7            6        0                           46439.0   
           8            0        1                           95647.0   
           9            1        1                           95983.0   
           10           2        1                           91099.0   
           11           3        1                           94743.0   
           12           4        1                           93911.0   
           13           5        0                           65757.0   
           14           6        0                           43366.0   
           15           0        1                           43692.0   
           16           1        1                          100558.0   
           17           2        1                           92887.0   
           18           3        1                           95866.0   
           19           4        1                           98413.0   
           20           5        0                           73063.0   

                (02001) centro comercial santa fe  (02101) toberin  \
year month day                                                       
2018 9     1                               9480.0          16920.0   
           2                               3874.0           6